# Case Study

## Introduction

**Learning Objectives**
- Applying what we've learned to a real world dataset
- How to create a PyTorch MLP
- How to properly batch data
- How to create training and testing functions

In the previous sections we have been using **PyTorch** to explain fundamental principles of **MLPs**. However, much of what we have done has been wrapped into singular functions, and we were often working with very simple architectures. Here, we will work through one of the foundational machine learning problems: **handwritten digit classification**.

## MNIST MLP

We will use **PyTorch** to implement a multi-layer perceptron (MLP) for classifying the **MNIST dataset**. As mentioned in a previous section, MNIST consists of greyscale images of handwritten digits and is commonly used as a benchmark dataset for developing and evaluating machine learning models in image analysis.

When working with PyTorch, we do not need to implement every component of the network and training process from scratch. The framework provides a range of high-level modules and utilities that simplify the design, training and evaluation of neural networks. 

Specifically, the `nn` module allows us to define network layers in a clear and structured way.



KEY TERMS BOX

[`torch.nn`](https://pytorch.org/docs/stable/nn.html) contains all of the building blocks for computational neural networks.

### Neural net class

To create a neural network with **PyTorch** you first need to create a class that **subclasses** the `torch.nn.Module`. Subclassing essentially copies the blueprint of the class given. 

From this you will then need to define the *layers*, *activation functions* and *output layer*, and when they are called. There are some different stylistic ways of doing this and you may see alternative ways of approaching is as we progress through this module.

We have defined a **MLP** below, have a study of it and see what you can understand before we break it down after:

In [ ]:
# Use the nn module to create an MLP to classify MNIST, with 3 layers:

# Import the nn module from PyTorch
import torch.nn as nn

# Define the MLP class:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # The forward pass is the sequence of operations 
        out = nn.Sequential(
            # flatten the image:
            nn.Flatten(),
            # apply the first linear layer:
            self.linear1,
            # apply the ReLU activation function:
            nn.ReLU(),
            # apply the second linear layer:
            self.linear2,
            # apply the ReLU activation function:
            nn.ReLU(),
            # apply the third linear layer:
            self.linear3,
            # apply the softmax activation function. 
            nn.Softmax(dim=1)
        )(x)
        return out


### Breakdown

In [ ]:
# Setting up the class
class MLP_demo(nn.Module): # class objectname = MLP
    
    def __init__(self, input_size, hidden_size, output_size):

        super().__init__() # call the parents class constructor

        # Define the layers
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, output_size)

To create the class we first call `class MLP(nn.Module)` to ensure it has all the methods/functions from the `nn.Module`.

 The first method is `__init__` which all classes need. This is where you declare the variables that will take the size of the input neurons, number of neruons in the hidden layers, and output size (number of classes). It is also where you declare the architecture of your **MLP**.

In `__init__` you first call `super().__init__`, which calls the parent class's (the one we subclassed) constructor, which means it is properly created. Without this you will not get all the functionailty of the `nn.Module`.

Next we can define the layers. Here we have a simple **3 layer** MLP:
- linear1 = **Input layer**
- linear2 = **Single hidden layer**
- linear3 = **output layer**

Each one is declared by calling `self` with `.layer_name`. The object `self` references the class itself and is a way of attaching *attributes* or functions to class. To this we assign our standard *linear* layer with [`nn.linear`](https://docs.pytorch.org/docs/stable/generated/torch.nn.Linear.html), connecting our layers. Due to this we must pass the size of the input and output layer.

In [ ]:
def forward(self, x):
    # The forward pass is the sequence of operations 
    # that the input x goes through in the network. 
    # In this case, the input goes through 3 linear layers,
    # where a ReLU activation function is applied to the 
    # output of the first two layers, before the output is
    # passed through a softmax activation function:
    out = nn.Sequential(
        # flatten the image:
        nn.Flatten(),
        # apply the first linear layer:
        self.linear1,
        # apply the ReLU activation function:
        nn.ReLU(),
        # apply the second linear layer:
        self.linear2,
        # apply the ReLU activation function:
        nn.ReLU(),
        # apply the third linear layer:
        self.linear3,
        # apply the softmax activation function. 
        # This is necessary because the output 
        # of the network is a probability distribution, 
        # and we want the the most probable class to be 
        # 1 and the rest to be 0:
        nn.Softmax(dim=1)
    )(x)
    return out

With the layers defined we need to declare the order of the **forward pass** with a `forward()` method.

Inside `forward()` we use the [`nn.Sequential`](https://docs.pytorch.org/docs/stable/generated/torch.nn.Sequential.html) class, which allows us to define a neural network as an *ordered sequence of fully-connected layers*. Here, we process the data with data with `flatten()`, pass it through the first layer, and apply the activation function. We then pass the result through the second layer and finally to the final layer for the prediction.

Finally, the predictions *logits* are passed to the [`nn.Softmax`](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) function to convert them into probabilities.

### Model parameters

In the previous section we manually worked out a hyopthetical models number of parameters. If we'd like to do that again for this model we can use this piece of code to calculate it for us:

In [ ]:
# The trainable parameters

# Create an instance of the MLP class with our input and output size
model = MLP(input_size=784, hidden_size=512, output_size=10)

total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

Even for a simple **MLP** our training parameters are already at approxiamtely **670 thousand**.

The figure below gives an insight into the density of a multi-layer model where **784** input features are propagated throughout the model.

![Fig. 7: Showing the pixel-wise computing of an image through a trained model.](./images/Fig_pixels.png)

## Dataloaders

As the MNIST dataset is a machine learning benchmark, it is available in PyTorch for automatic download through the [`datasets`](https://pytorch.org/vision/stable/datasets.html) module. 

PyTorch also has a [`DataLoader`](https://pytorch.org/docs/stable/data.html) class that is used to load the data in batches for training and testing. The **DataLoader** class expects a PyTorch *Dataset* object, so you must first create a *Dataset* object. When downloading one from PyTorch it already comes in this format, however when working with your own data, you will need to format it yourself. The other arguments are  batch size, the number of samples per batch to load, and whether to shuffle the data.

It is good practice to normalise your data before training, even with image pixels. This ensures that no data feature dominates the others due to having very large values, which can make training unstable or slow.

For image data, a simple approach is to use mean and std of `0.5, 0.5`, which rescales pixel values from `[0, 1]` to `[-1, 1]`. However, it is better practice to normalise using the actual mean and std calculated from your training dataset, as we have done here with MNIST: **mean=0.1307, std=0.3081**. This centres the data around 0 and scales it to have unit variance, which helps gradient descent converge more efficiently and prevents activation functions from saturating.

In [ ]:
from torch.utils.data import DataLoader 
from torchvision import datasets, transforms

# The number of samples that will be propagated through the network at one time.
BATCH_SIZE_TRAIN = 64
BATCH_SIZE_TEST = 1000


# Normalise the data to have mean 0 and std 1 using datasets statisitcs
transform = transforms.Compose([
                          transforms.ToTensor(),    # convert the image to a tensor
                          transforms.Normalize((0.1307,), (0.3081,)) # normalise the image
                        ])

# Define a training data loader

train_loader = DataLoader(
    datasets.MNIST( 'data', 
                    train=True, 
                    download=True,
                    transform = transform
                    ),
    batch_size=BATCH_SIZE_TRAIN, 
    shuffle=True # shuffling the data order can train a more robost model 
    ) 

# Define a test data loader
test_loader = DataLoader(
    datasets.MNIST( 'data', 
                    train=False, 
                    download=True,
                    transform = transform
                    ),
    batch_size=BATCH_SIZE_TEST, 
    )

Now we have our data, lets have a quick look at it.

In [ ]:
import matplotlib.pyplot as plt

# Define a function to plot the images
def plot_images(images, labels):
    """
    Display a 4x4 grid of MNIST images with their corresponding labels.
    
    :param images: Batch of image tensors to display. Each image should be
                   either 28x28 or flattened to 784 pixels.
    :type images: torch.Tensor
    :param labels: Corresponding labels for each image (digit 0-9).
    :type labels: torch.Tensor
    
    :returns: None (displays the plot using plt.show())
    """
    fig, axes = plt.subplots(4, 4)
    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i].reshape(28, 28), cmap='binary')
        ax.set(xticks=[], yticks=[], xlabel=int(labels[i]))
        # make a white background for the text
        ax.xaxis.label.set_color('white')
        # increase the vertical distance between the subplots:
        fig.subplots_adjust(hspace=0.5)
    plt.show()


# Plot a few of the MNIST images with their labels:
images, labels = next(iter(train_loader))
plot_images(images, labels)

That was fairly easy to implement due to PyTorch's built-in functions. However, we will be typically working with our own unique datasets, so it is worth understanding making your own `Datasets`. We will demonstrate this with a pandas dataset.

Like with the **MLP** we will *subclass* the `Dataset` class, from the `torch.utils.data` module. We need to implement **three** methods:

1. `__init__`: Where we pass the features (X) and targets (y) to the dataset. If you were to perform any other processes, such as loading from a file path, transforming, or normalising, you would do it here.

2. `__len__`: Returns the number of samples in the dataset. When this function is called, it should provide the length of the dataset, which is typically the total number of samples that are available to be returned. This is useful for understanding the size of the dataset and is used by DataLoader to determine how many batches are needed to iterate over the entire dataset.

3. `__getitem__`: Returns a single sample from the dataset.This method is essential for indexing and iterating through the dataset. It can include the logic for data loading, preprocessing, and transformation, returning the data sample (and possibly its label) in the format required for model training.

In [ ]:
from torch.utils.data import Dataset
from torch import LongTensor, FloatTensor

class SimpleTabularDataset(Dataset):
    """Simple dataset for tabular data."""
    
    def __init__(self, X, y):
        """
        X: Your features (pandas DataFrame or numpy array)
        y: Your targets (pandas Series or numpy array)
        """
        super().__init__()
        
        # Pandas has .values, numpy does not
        if hasattr(X, 'values'): # pandas
            X = X.values
            y = y.values
        else: # numpy
            X = np.array(X)
            y = np.array(y)
        
        # Convert to torch tensors
        self.X = FloatTensor(X)
        self.y = LongTensor(y)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

We then create the custom dataset class by calling it with the correct data. This can then be passed to the `DataLoader`.

In [ ]:
import pandas as pd

# Load the data
df = pd.read_csv('./data/patients.csv')
print(df.head(3))

In [ ]:
X = df.drop('Smoker', axis=1)  # Features
y = df['Smoker']               # Target

# Create dataset
dataset = SimpleTabularDataset(X, y)

# Create DataLoader
tabular_loader = DataLoader(dataset, batch_size=32, shuffle=True)

## The Training Function

Finally, we can now define our functions to *train* and *test* the model.

For the train function we will need it to accept as arguments:
- the model
- the device where the training occurs in memory (cpu or gpu)
- the data loader
- the number of training epochs

In [ ]:
import torch.nn.functional as F

# Define a function to train the model:

def train(model, device, train_loader, optimiser, epoch):
    """
    Train a neural network model for one epoch.
    
    :param model: The neural network model to train. Must have a forward() method
                  and be compatible with PyTorch's training paradigm.
    :type model: torch.nn.Module
    :param device: The device (CPU or GPU) on which to perform training computations.
                   Typically created using torch.device("cpu") or torch.device("cuda").
    :type device: torch.device
    :param train_loader: DataLoader containing the training dataset. Yields batches
                         of (data, target) tuples during iteration.
    :type train_loader: torch.utils.data.DataLoader
    :param optimiser: The optimization algorithm used to update model parameters.
                      Common choices include SGD, Adam, etc.
    :type optimiser: torch.optim.Optimizer
    :param epoch: Current epoch number (used for progress reporting).
    :type epoch: int
    
    :returns: None (prints training progress to console)
    """
    model.train() # set the model to training mode

    for batch_idx, (data, target) in enumerate(train_loader):

        # Load the data in and its target variables
        data, target = data.to(device), target.to(device) # send the data to the device

        # Complete a forward pass and calculate the loss
        optimiser.zero_grad() # zero the gradients
        output = model(data) # forward pass
        loss = F.nll_loss(output.log(), target) # calculate the loss

        loss.backward() # backward pass
        optimiser.step() # update the weights

        # Print the training loss
        if batch_idx % 500 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]')
            print(f'Loss: {loss.item():.6f}')

For clarity with the previous teaching materials, when creating our neural net architecture we placed the **softmax** activation function after the final layer. The loss is then calculated using [`F.nll_loss()`](https://docs.pytorch.org/docs/stable/generated/torch.nn.functional.nll_loss.html) which expects log-probabilities.

However, PyTorch has a [Cross Entropy](https://docs.pytorch.org/docs/stable/generated/torch.nn.functional.cross_entropy.html) function that calculates **softmax** and the **NLL Loss** all together. Therefore, we can call that instead, albeit after removing the **softmax** function from the forward pass in our model.

```
def train(model, device, train_loader, optimiser, epoch):

    model.train() # set the model to training mode

    for batch_idx, (data, target) in enumerate(train_loader):

        ----- SAME CODE -----

        optimiser.zero_grad() # zero the gradients
        output = model(data) # forward pass

        loss = F.cross_entropy(output, target) # Calculate the softmax and loss all in one go

        loss.backward() # backward pass
        optimiser.step() # update the weights

        ----- SAME CODE -----
```

## Testing Function

For the test function we repeat much of the training function, with the exception that we are **not updating the *parameters***. This means that we don't need to perform the *backwards pass* or update the *weights*. Addtionally, when you are only getting a prediction from the model, you will always set it to evaluation mode with `.eval()`. This is an important step when your model has layers/parts that perform differently during training and inference. For example, dropout layers, which randomly turn off neurons during training will be switched off so that the model is once again fully connected.

In [ ]:
import torch

# Define a function to test the model
def test(model, device, test_loader):
    """
    Evaluate a trained neural network model on the test dataset.
    
    :param model: The trained neural network model to evaluate. Must have a forward()
                  method and be compatible with PyTorch's evaluation paradigm.
    :type model: torch.nn.Module
    :param device: The device (CPU or GPU) on which to perform evaluation computations.
                   Typically created using torch.device("cpu") or torch.device("cuda").
    :type device: torch.device
    :param test_loader: DataLoader containing the test dataset. Yields batches of
                        (data, target) tuples during iteration.
    :type test_loader: torch.utils.data.DataLoader
    
    :returns: None (prints test results to console including average loss and accuracy)
    """
    model.eval() # set the model to evaluation mode

    # Set variables to collect cumulative loss and correct predictions
    test_loss = 0
    correct = 0

    with torch.no_grad(): # don't calculate the gradients

        for data, target in test_loader:

            data, target = data.to(device), target.to(device) # send the data to the device

            # Complete a forward pass and calculate the loss
            output = model(data) # forward pass
            test_loss += F.nll_loss(output.log(), target, reduction='sum').item() # sum up the batch loss
            
            # Get the top prediction and check if correct
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max probability
            correct += pred.eq(target.view_as(pred)).sum().item() # Check if the prediction is correct

    # Calculate the average loss and accuracy
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)

    print('--------Evaluation--------\n')
    print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)')
    print('--------Next Epoch--------\n')

## Running Training

Before we can run the *training* and *testing* functions in a loop, we must first declare the **device** the training will run on. 

**Graphics Processing Units** (GPUs) are the hardware that unlocked *deep learning*, they can process the matrix calculations at a much faster rate than a standard **Central Processing Unit** (CPU) typically found in your computer. Therefore, if you have the access to GPUs then you're training and testing will be a fraction of the time. With *PyTorch* you must move your data and processing to the **GPU**, or in other words more it to the **device** you want.

First the **device** must be defined using the `torch.device()` function, and secondly the model must be moved to the **device** with its `.to()` method:

In [ ]:
# Define the device

# Check if GPU is available
if torch.cuda.is_available():
    # if there are multiple GPUs, choose the first one
    device = torch.device(f"cuda:0")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(f'Device name: {torch.cuda.get_device_name(0)}')
else:
    print("No GPU detected! Falling back to CPU")
    # If the GPU is not available, use the CPU
    device = torch.device("cpu")

# Create an instance of the MLP class
model = MLP(input_size=784, hidden_size=512, output_size=10)

# Move the model onto the device (only needed with GPU)
model.to(device);

NOTE BOX

Throughout this module you will see data moved from the **device** to the **CPU** through the chained methods `.detech().cpu()`. This removes the data from the current computational graph and onto the **CPU**, which is more efficient when not working with model itself. For example, when you wish to score or visualise the output.

We can now run our training loop. For demonstrative purposes, we will train it for a few epochs, only. In a real-world application, however, we would typically train the model over several hundreds of epochs, or more.

In [ ]:
# Imports
import torch.optim as optim

# Define the optimiser (SGD)
optimiser = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# Train the model for 5 epochs
for epoch in range(5):

    train(model, device, train_loader, optimiser, epoch)
    
    test(model, device, test_loader)

We can now see how our trained model performs on the test set.

In [ ]:
from numpy import array

# We can now use the model to make predictions on the test set:

# get the first batch of test images
images, labels = next(iter(test_loader))

# plot the images
plot_images(images, labels)

# get the model predictions
preds = model(images.to(device))

# get the index of the max log-probability
preds = preds.argmax(dim=1, keepdim=True)

# print the predictions

# squeeze removes single-dimensional entries from the shape of an array
predicted_labels_list = preds.squeeze().detach().cpu().tolist() 
print('Predicted labels:', predicted_labels_list[:16])

# print the actual labels
print('Actual labels:   ', labels[:16].tolist())

# Accuracy
accuracy = 100*(sum(array(predicted_labels_list) == array(labels.tolist())) / len(predicted_labels_list))
print(f'Accuracy: {accuracy} %')

With an accuracy score of around 97.6%, the model is performing very respectably. Training over only a few epochs led to the model only misclassifying 24 out of the 1000 test images we gave it.

NOTE BOX

In the above code and throughout this module you will see `.squeeze()` used, typically on data coming from the model or dataloader. For *PyTorch* tensors they often have an extra dimension of size 1 added to the start or end, for their internal processing. `.squeeze()` will remove any dimensions of size 1.

For example `preds` has a shape of `[1000, 1]` but we want `[1000]`. Using `.squeeze()` will remove the extra dimension of size 1 from the end..

## Summary

In this lesson, we examined how a trained neural network is evaluated and used beyond the training phase. Using a **multi-layer perceptron** applied to handwritten digit recognition, we explored how model predictions are generated through a *forward pass*, how performance is *quantified* using **accuracy**, and why **evaluation** *must be carried out on data that the model has not seen* during training. We also introduced the concept of **inference**, where a trained model is applied to new, unlabelled data. Together, these ideas complete the end-to-end picture of training, testing and deploying a simple neural network, and provide a foundation for working with more advanced models.